In [1]:
import boa
from utils import deploy_pool, deploy_infra, get_balances
from IPython.display import display, HTML

## **Table of Content**

1. [Infrastrucutre and Pool Setup](#infrastrucutre-and-pool-setup)
2. [Adding Liquidity](#exchanging-tokens)
3. [Calculating LP Tokens](#3-calculating-lp-token-amount)
4. [Calculating Fee](#4-calculate-fee-whenn-adding-liquidity)

### **1. Infrastrucutre and Pool Setup**

Before we can exchange tokens, we need to deploy the twocrypto-ng infrastructure and deploy a pool. Afterwards, we grant approval to the pool.

In [2]:
# deploying the infrastructure for twocrypto-ng pools

infra = deploy_infra()
factory = infra["factory"]
trader = infra["trader"]
coin_a = infra["coin_a"]
coin_b = infra["coin_b"]
fee_receiver = infra["fee_receiver"]

# lets deploy a pool 
pool = deploy_pool(factory, coin_a, coin_b)
pool = infra["impl"].at(pool)

/Users/mo/Documents/CurveDocs/boa/lib/python3.12/site-packages/vyper/ast/folding.py:153: UserWarning: ZERO_ADDRESS is deprecated. Please use `empty(address)` instead.
  warnings.warn(f"{name} is deprecated. Please use `{replacement}` instead.")
/Users/mo/Documents/CurveDocs/boa/lib/python3.12/site-packages/vyper/semantics/types/subscriptable.py:115: UserWarning: Use of large arrays can be unsafe!
  warnings.warn("Use of large arrays can be unsafe!")


In [3]:
# grant max approval

with boa.env.prank(trader):
    for coin in [coin_a, coin_b]:
        coin.approve(pool, 2**256-1)


print(coin_a._storage.allowances.get())
print(coin_b._storage.allowances.get())

{'0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B': {'0x5Bac22268eecccDF2fa09D03eC0D0fDbA2848ba2': 115792089237316195423570985008687907853269984665640564039457584007913129639935}}
{'0x1fb0aF040B7bbA2A6f69e77DA9C0dCb7785b3A3B': {'0x5Bac22268eecccDF2fa09D03eC0D0fDbA2848ba2': 115792089237316195423570985008687907853269984665640564039457584007913129639935}}


---

### **2. Adding Liquidity**

- #### `add_liquidity`

    Function to add liquidity to the pool and mint the corresponding LP tokens.

    Returns: Amount of LP tokens received (`uint256`).

    Emits: `AddLiquidity`

    | Input            | Type                | Description                                           |
    | ---------------- | ------------------- | ----------------------------------------------------- |
    | `amounts`        | `uint256[N_COINS]`  | Amount of each coin to add. We define `amount_a` as `Pool.coins(0)`, `amount_b` as `Pool.coins(1)`. |
    | `min_mint_amount`| `uint256`           | Minimum amount of LP tokens to mint.                  |
    | `receiver`       | `address`           | Receiver of the LP tokens; defaults to `msg.sender`.  |

First, we add some liquidity in a balanced proportion and check the coin balances before and after:

In [4]:
amounts_a = 10**20  # Pool.coins(0)
amounts_b = 10**20  # Pool.coins(1)
min_mint_amount = 99
receiver = trader

print("BEFORE ADDING ANY LIQUIDITY:")
display(HTML(get_balances(trader, pool, coin_a, coin_b).to_html(index=False)))

with boa.env.prank(trader):
    pool.add_liquidity([amounts_a, amounts_b], min_mint_amount, receiver)
    
# we check again after adding liquidity
print("===============================")
print(" ")
print("BALANCES AFTER ADDING LIQUIDITY")
display(HTML(get_balances(trader, pool, coin_a, coin_b).to_html(index=False)))

print(coin_a.balanceOf(fee_receiver))
print(coin_b.balanceOf(fee_receiver))

BEFORE ADDING ANY LIQUIDITY:


Asset,Balance
Trader: Balance A,100000.0
Trader: Balance B,100000.0
Trader: LP Tokens,0.0
Pool: Balance A,0.0
Pool: Balance B,0.0


 
BALANCES AFTER ADDING LIQUIDITY


Asset,Balance
Trader: Balance A,99900.0
Trader: Balance B,99900.0
Trader: LP Tokens,100.0
Pool: Balance A,100.0
Pool: Balance B,100.0


0
0


As shown, 100 tokens of each coin was added. In exchange we received 100 LP tokens.

Now lets add liquidity again, but this time in an inbalanced proportion: 10 coin_a and 100 coin_b. But before that, lets try out the `calc_token_amount` function. This function does not alter the state. It is used to calculate the LP tokens to be minted or burned when depositing or removing coins.

---

### **3. Calculating LP Token Amount**

- #### `calc_token_amount(amounts: uint256[N_COINS], deposit: bool) -> uint256:`

    Function to calculate the LP tokens to be minted or burned for depositing or removing `amounts` of coins. This method takes fees into consideration.

    Returns: Amount of LP tokens deposited or withdrawn (`uint256`).

    | Input            | Type                | Description                                           |
    | ---------------- | ------------------- | ----------------------------------------------------- |
    | `amounts`        | `uint256[N_COINS]`  | Amount of each coin to add/remove. We define `amount_a` as `Pool.coins(0)`, `amount_b` as `Pool.coins(1)`. |
    | `deposit`        | `bool`              | `True` for depositing, `False` for removing            |

Before actually adding liquidity imbalanced, lets calculate the lp tokens we receive when doing so:

In [5]:
amounts_a = 10**19  # Pool.coins(0)
amounts_b = 10**20  # Pool.coins(1)

with boa.env.prank(trader):
    lp_token = pool.calc_token_amount([amounts_a, amounts_b], True)

print(f"LP Tokens to receive: {lp_token / 1e18}")

LP Tokens to receive: 48.47325226409857


Adding 10^19 of coin_a and 10^20 of coin_b *should* give us 48473252264098567798 LP tokens. 

Now, lets actually add liquidity to see how much we receive:

In [6]:
amounts_a = 10**19
amounts_b = 10**20
precision_a = pool.precisions()[0]
precision_b = pool.precisions()[1]
pool_balance_a = pool.balances(0)
pool_balance_b = pool.balances(1)
xp_a = pool_balance_a * precision_a
xp_b = pool_balance_b * precision_b

pool.calc_token_fee([amounts_a, amounts_b], [xp_a, xp_b])

10736363

In [7]:
amounts_a = 10**19  # Pool.coins(0)
amounts_b = 10**20  # Pool.coins(1)
min_mint_amount = 99
receiver = trader

# we check the balances before providing liquidity
print("BEFORE ADDING ANY LIQUIDTY:")
display(HTML(get_balances(trader, pool, coin_a, coin_b).to_html(index=False)))

with boa.env.prank(trader):
    pool.add_liquidity([amounts_a, amounts_b], min_mint_amount, receiver)

# we check again after adding liquidity
print("===============================")
print("")
print("AFTER ADDING LIQUIDTY:")
display(HTML(get_balances(trader, pool, coin_a, coin_b).to_html(index=False)))

BEFORE ADDING ANY LIQUIDTY:


Asset,Balance
Trader: Balance A,99900.0
Trader: Balance B,99900.0
Trader: LP Tokens,100.0
Pool: Balance A,100.0
Pool: Balance B,100.0



AFTER ADDING LIQUIDTY:


Asset,Balance
Trader: Balance A,99890.000000
Trader: Balance B,99800.000000
Trader: LP Tokens,148.473252
Pool: Balance A,110.000000
Pool: Balance B,200.000000


As shown, the trader receives the amount calculated in `calc_token_fee`.

### **4. Calculate Fee Whenn Adding Liquidity**

- #### `calc_token_fee(amounts: uint256[N_COINS], xp: uint256[N_COINS]) -> uint256:`

    Function to calculate the charged fee on `amounts` when adding liquidity.

    Returns: Charged fee (`uint256`).

    | Input            | Type                | Description                                           |
    | ---------------- | ------------------- | ----------------------------------------------------- |
    | `amounts`        | `uint256[N_COINS]`  | Amount of each coin to add. `amount_a` is for `Pool.coins(0)`, `amount_b` `Pool.coins(1)`. |
    | `xp`| `uint256[N_COINS]`               | Pool balances multiplied by the coin precisions.      |


Minimum fee when adding liquidity is 100000, this occurs when the pool is perfectly balanced and liquidity is added in a balanced portion?

In [8]:
amounts_a = 10**19
amounts_b = 10**20
precision_a = pool.precisions()[0]
precision_b = pool.precisions()[0]
pool_balance_a = pool.balances(0)
pool_balance_b = pool.balances(1)
xp_a = pool_balance_a * precision_a
xp_b = pool_balance_b * precision_b

pool.calc_token_fee([amounts_a, amounts_b], [xp_a, xp_b])

18487938

One might try to set `pool_balance_a` and/or `pool_balance_b` to 0 to calculate the fee when adding liquidity to an empty pool: This does not work (dividing with 0 is not possible) for the sole reason that when adding liquidity, the fee is calculated based on the pool balances AFTER the tokens have been transfered into the pool.

---

Lets check the fees. We need to call `remove_liquidity_one_coin` as this function calls the internal `_claim_admin_fees` function. The fees that occur for removing single sided is not being cosidered in the fee claiming, as the fee claiming is the first "action" in the function.

In [9]:
print(coin_a.balanceOf(fee_receiver))
print(coin_b.balanceOf(fee_receiver))

with boa.env.prank(trader):
    pool.remove_liquidity_one_coin(10**18, 0, 0, trader)

print(coin_a.balanceOf(fee_receiver))
print(coin_b.balanceOf(fee_receiver))

0
0
49858304293238500
90651462351342729


In [10]:
fee = 10736363

49858304293238500
90651462351342729

90651462351342729